# Analyzing Netflow Data with xGT

This sample script loads raw NetFlow data in an xGT graph structure and queries for a graph pattern.

The dataset used is from the CTU-13 open source project:
https://mcfp.weebly.com/the-ctu-13-dataset-a-labeled-dataset-with-botnet-normal-and-background-traffic.html

Raw data example:

```
StartTime   SrcAddr       DstAddr       State  sTos  dTos  TotPkts  TotBytes
2011/08/16  147.32.86.58  77.75.73.9    SR_A   0.0   0.0   3        182
2011/08/16  147.32.3.51   147.32.84.46  S_RA   0.0   0.0   4        124
```

This notebook follows this sequence of steps:

1. Setup python environment
2. Read the input netflow file
3. Create graph schema
4. Upload the data to the Trovares xGT server
5. Run a query

## 1. Setup Python Environment

  - Create Trovares xGT setup/connection
  - Register with Graphistry

In [1]:
import numpy as np
import pandas as pd
import sys
import csv
import re
import os
import xgt

# For cloud instances, replace the localhost with the instance's IP address or use ssh tunneling
server = xgt.Connection(host='localhost', auth=xgt.BasicAuth('xgtd'))
server.set_default_namespace('ctu13')
xgt.__version__

'1.14.1'

In [2]:
import graphistry

# To specify Graphistry account & server, use:
# graphistry.register(api=3, username='...', password='...', protocol='https', server='hub.graphistry.com')
# For more options, see https://github.com/graphistry/pygraphistry#configure
import getpass
graphistry.register(api=3, username='your_username', password=getpass.getpass(),
                    protocol='https', server='hub.graphistry.com')

········


## 2. Read the input netflow file

- Read the input netflow file from the file system into the pandas Dataframe.
- Do data transformations to align with Trovares xGT

In [3]:
%%time
def cleanup_data(x):
  if x == '':
    return pd.NA
  elif isinstance(x, str):
    return int(x, 16)
  return x

# Ingest data, translating datetime format to ISO standard.
input_filename = "https://mcfp.felk.cvut.cz/publicDatasets/CTU-Malware-Capture-Botnet-46/detailed-bidirectional-flow-labels/capture20110815-2.binetflow"
from datetime import datetime
ctu_date_parser = lambda x: datetime.strptime(x, '%Y/%m/%d %H:%M:%S.%f').strftime("%Y-%m-%dT%H:%M:%S.%f")
df = pd.read_csv(input_filename, parse_dates=['StartTime'], date_parser=ctu_date_parser, converters={"Sport": cleanup_data, "Dport": cleanup_data})

CPU times: user 1.95 s, sys: 168 ms, total: 2.12 s
Wall time: 18.6 s


In [4]:
df.sample(4)

,StartTime,Dur,Proto,SrcAddr,Sport,Dir,DstAddr,Dport,State,sTos,dTos,TotPkts,TotBytes,SrcBytes,Label
23799,2011-08-15 16:47:51.845660,2.502171,tcp,66.249.66.10,287316,->,147.32.87.36,128,FSPA_FSPA,0.0,0.0,10,1375,757,flow=To-Background-CVUT-WebServer
96292,2011-08-15 17:05:01.103720,1.235905,tcp,147.32.84.21,331797,->,74.125.39.108,2453,FSPA_FSRPA,0.0,0.0,29,3049,1473,flow=Background-google-pop
59210,2011-08-15 16:56:02.617817,0.000538,udp,147.32.85.47,333719,<->,147.32.80.9,83,CON,0.0,0.0,2,292,74,flow=To-Background-UDP-CVUT-DNS-Server
9480,2011-08-15 16:44:53.022852,0.000000,icmp,147.32.84.59,771,->,147.32.80.9,14303,URP,0.0,NaN,1,70,70,flow=Background-Attempt-cmpgw-CVUT


## 3. Create graph schema

In [5]:
# Create a vertex frame on the xGT server.
server.drop_frame('Netflow')
server.drop_frame('IP')
ip = server.create_vertex_frame(
    name = 'IP',
    schema = [['IPAddr', xgt.TEXT]],
    key = 'IPAddr',
)

In [6]:
# Create a netflow edge frame on the xGT server.
server.drop_frame('Netflow')
netflow = server.create_edge_frame(
    name = 'Netflow',
    schema = [
        ['StartTime', xgt.DATETIME], ['Dur', xgt.FLOAT], ['Proto', xgt.TEXT], ['SrcAddr', xgt.TEXT],
        ['Sport', xgt.INT], ['Dir', xgt.TEXT], ['DstAddr', xgt.TEXT], ['Dport', xgt.INT],
        ['State', xgt.TEXT], ['sTos', xgt.FLOAT], ['dTos', xgt.FLOAT],['TotPkts', xgt.INT],
        ['TotBytes', xgt.INT], ['SrcBytes', xgt.INT], ['Label', xgt.TEXT],
    ],
    source = ip,
    target = ip,
    source_key = 'SrcAddr',
    target_key = 'DstAddr', 
)

## 4. Upload the data to the Trovares xGT server


In [7]:
%%time
# Note that the graph vertices containing IP addresses will be automatically created in the
# xGT server for any IP address mentioned as either source or target of a netflow edge.
netflow.insert(df)
print(f"IP count: {ip.num_rows:,}")
print(f"Netflow record (edges) count: {netflow.num_rows:,}")

IP count: 41,658
Netflow record (edges) count: 129,832
CPU times: user 48.3 ms, sys: 21.4 ms, total: 69.7 ms
Wall time: 235 ms


In [8]:
# Show memory footprint
max_memory = server.max_user_memory_size
print(f"Memory footprint: {max_memory - server.free_user_memory_size:,.3f} GiB used out of {max_memory:,.3f} GiB available.")

Memory footprint: 0.013 GiB used out of 32.000 GiB available.


## 5. Run a query

Run a `MATCH` query looking for a two-cycle that satisfy a bunch of constraints:

- The two edges are ordered by time.
- The durations are increasing throughout the path; the second edge has a much larger duration than the first.
- The two edges have these *OSI transport layer* protocols:  (tcp, icmp)


In [9]:
%%time
job = server.run_job("""
    MATCH (a)-[e1]->(b)-[e2]->(a)
    WHERE e1.StartTime <= e2.StartTime
      AND e1.Dur < (e2.Dur / 10)  // e2 duration at least 10 times longer than e1
      AND e1.Proto = 'tcp'
      AND e2.Proto = 'icmp'
    RETURN
      a.IPAddr AS A, e1.StartTime AS timestamp1, e1.Dur AS dur1,
      b.IPAddr AS B, e2.StartTime AS timestamp2, e2.Dur AS dur2
""")

result_set = job.get_data(format='pandas')
print("Number of results: " + str(job.total_rows))
print(f"Total number of visited edges: {job.total_visited_edges:,}")

Number of results: 38
Total number of visited edges: 69,854
CPU times: user 5.86 ms, sys: 743 µs, total: 6.6 ms
Wall time: 15.6 ms


In [10]:
# Uncomment to see the actual answers in a pandas frame
# result_set

In [11]:
## 6. Plot answers with Graphistry
g = graphistry.edges(result_set, 'A', 'B')

In [12]:
g.settings(url_params={'dissuadeHubs':True, 'strongGravity': True}).plot()

<footer>Copyright &copy; 2021-2023 Trovares Inc</footer>